# Intro to postgreSQL and psycopg2

**Cameron Bronstein - _July, 2019_**



#### Dependencies

- `psycopg2`: version 2.7.7
- `postgreSQL`: [version 11.x](https://www.enterprisedb.com/downloads/postgres-postgresql-downloads)

#### Documentation
- [postgreSQL 11](https://www.postgresql.org/docs/11/index.html)

#### Data Sources
- [West Virginia Counties](https://www.westvirginia-demographics.com/counties_by_population)
- [Opioid Use Data](https://www.washingtonpost.com/graphics/2019/investigations/dea-pain-pill-database/?utm_term=.b945f1d2a341#download-resources)


In [ ]:
# in case psycopg2 is not installed...
# !pip install psycopg2-binary --user

In [ ]:
# a few imports
import psycopg2
import pandas as pd

## Connecting to Postgres DB

**Arguments:**
- `host = localhost`
- `dbname` = **your choice of db**
- `user` = **your postgres username**

In [ ]:
# establishin a connection
connection = psycopg2.connect(host = "localhost", dbname = "dbname", user = "username")

In [ ]:
# creating the cursor - this is a python object that executes all of your SQL commands
cursor = connection.cursor()

### Create a table skeleton in the PG database

**Steps:**
- title
- column names
- column data types
- primary key

In [ ]:
command = """
CREATE TABLE west_virginia(
transaction_date date,
reporter_name varchar,
reporter_address varchar,
buyer_name varchar, 
buyer_category varchar,
buyer_address varchar,
buyer_county varchar,
product_name varchar,
drug_name varchar,
dosage_unit float,
quantity float,
base_wt_gm float,
labeler_name varchar,
company_name varchar)
"""

cursor.execute(command)

### Get the data

In [ ]:
# filling the empty table with data!

with open("./data/west_virginia_for_lesson.tsv") as f:
    
    # why do we use next, here?
    next(f)
    
    # here is cursor, again
    cursor.copy_from(f, 'west_virginia', sep='\t')

# this establishes the end of a command block    
connection.commit()

In [ ]:
# confirming that we have data

command = """
SELECT COUNT(*)
FROM west_virginia
"""

cursor.execute(command)

cursor.fetchall()

### West Virgina Counties // Relational Databases 

In [ ]:
counties = pd.read_csv('./data/west_virginia_pop_by_county.csv')
counties.head()

In [ ]:
counties.head()

<details>
  <summary>How would we create a counties table?</summary>
  
  ```
  CREATE TABLE counties(
  County varchar,
  Population integer,
  Rank integer)
  
  with open("./data/west_virgina_pop_by_county.csv") as f:
      next(f)
    
      cursor.copy_from(f, 'counties', sep=',')

  connection.commit()
  ```
</details>

In [ ]:
command = """

"""

cursor.execute(command)

In [ ]:
# fill the counties table with data!


### SQL portion

- **First, an aside:**
    - Case study: [Leavenworth County, Kansas](https://www.kcur.org/post/leavenworth-county-kansas-may-not-be-catastrophic-opioid-hotspot-new-data-appear-show#stream/0). This is important to being a data scientist, too!

In [ ]:
# dislapy the column names from the west_virginia table 

command = """
SELECT column_name, data_type 
FROM information_schema.columns
WHERE table_name = 'west_virginia';
"""

# this is the necessary syntax for psycopg2

cursor.execute(command)
cursor.fetchall()
connection.commit()

#### About psycopg2:

- **Take notice of the output of `cusor.fetchall()`**
    - It returns a list of tuples, with each element in the tuple ordered by the information specified in our query.
    - For the purpose of this lesson, this data format might not make sense, but it is important to understand as future Data Engineers and Data Scientists. This format might be more useful in an ETL (extract, transform, load) pipeline than running the data through a Pandas DataFrame
    
- For the rest of this lesson, we will run our queries through `pandas` as you are likely familiar with from previous lessons and assignments. 
    - **But** our connection to the `postgres` database still relies on `psycopg2`

### Query 1

<details>
  <summary>List the top 10 opioid buyers by grams purchased:</summary>
  
  ```
SELECT buyer_name as buyer,
       SUM(base_wt_gm) as total_ordered
FROM west_virginia
GROUP BY buyer_name
ORDER BY total_ordered DESC
LIMIT 10;
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 2

<details>
  <summary>List the top 10 opioid buyers by number of transactions, compared to total grams ordered:</summary>
  
  ```
SELECT buyer_name as buyer,
       SUM(base_wt_gm) as total_ordered
       COUNT(buyer_name) as number_transactions
FROM west_virginia
GROUP BY buyer_name
ORDER BY number_transactions DESC
LIMIT 10;
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 3

<details>
  <summary>List the top 10 counties by number of transactions:</summary>
  
  ```
SELECT buyer_county as buyer_name,
       COUNT(buyer_county) as number_transactions
FROM west_virginia
GROUP BY buyer_name
ORDER BY number_transactions DESC
LIMIT 10;
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 4

<details>
  <summary>How does this compare to total number grams purchased per county?</summary>
  
  ```
SELECT buyer_county as county, 
       count(buyer_county) as number_transactions,
       SUM(base_wt_gm) as total_grams_purchased
FROM west_virginia
GROUP BY county
ORDER BY total_grams_purchased DESC
LIMIT 10;
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 5

<details>
  <summary>What is the most common opioid product?</summary>
  
  ```
SELECT product_name as product,
       COUNT(product_name) as num_orders
FROM west_virginia
GROUP BY product
ORDER BY num_orders DESC
LIMIT 10;
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 6

<details>
  <summary>How does opioid purchases compare to county population size?</summary>
  
  ```
SELECT w.buyer_county as county, 
       count(w.buyer_county) as number_transactions,
       SUM(w.base_wt_gm) as total_grams_purchased,
       c.Population as population
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
ORDER BY c.Population DESC
LIMIT 20;
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 7

<details>
  <summary>How many counties have more opioid purchase transactions than people?</summary>
  
  ```
SELECT w.buyer_county as county, 
       count(w.buyer_county) as number_transactions,
       c.Population as population
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
HAVING count(w.buyer_county) > c.Population
ORDER BY number_transactions DESC
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 8

<details>
  <summary>What is their average number of opioids ordered by grams? Average number of pills?</summary>
  
  ```
SELECT w.buyer_county as county, 
       count(w.buyer_county) as number_transactions,
       c.Population as population,
       AVG(w.base_wt_gm) as average_grams_per_order,
       AVG(w.dosage_unit) as average_pills_per_order
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
HAVING count(w.buyer_county) > c.Population
ORDER BY average_grams_per_order DESC;
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 9

<details>
  <summary>What's the average number of pills per person per year?</summary>
  
  ```
SELECT w.buyer_county as county,
       c.Population as population,
       SUM(w.dosage_unit) as total_pills_purchased,
       SUM(w.dosage_unit) / 7 as pills_per_year,
       ((SUM(w.dosage_unit) / 7) / c.Population) as avg_pill_per_person 
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
ORDER BY avg_pill_per_person DESC;
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 10

<details>
  <summary>Pills per person - Statewide:</summary>
  
  ```
SELECT AVG(a.avg_pill_per_person) as statewide_average
FROM
(SELECT w.buyer_county as county,
       c.Population as population,
       SUM(w.dosage_unit) as total_pills_purchased,
       SUM(w.dosage_unit) / 7 as pills_per_year,
       ((SUM(w.dosage_unit) / 7) / c.Population) as avg_pill_per_person
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY w.buyer_county, c.Population
ORDER BY avg_pill_per_person DESC)
AS a;
  ```
</details>

In [ ]:
connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

### Query 11 - Group By Date

<details>
  <summary>Lets group opioid purchases by year:</summary>
  
  ```
SELECT to_char(transaction_date, 'YYYY') as year,
       COUNT(w.*) as number_transactions,
       SUM(w.dosage_unit) as total_pills
FROM west_virginia as w
JOIN counties AS c on c.County = w.buyer_county
GROUP BY year;
  ```
</details>

In [ ]:


connection.commit()

command = """

"""

dataframe = pd.read_sql(command, connection)
dataframe

## What else can you think of?
- Which suppliers were most prominant? Per year?
- County averages per year? How do we show this?
    - Queries can get complicated pretty quick!

## Command Line Tricks

- end all queries with `;`
- `\timing` to print query execution times in CLI.
- `\d+` to print database information
- `\d+ table_name` to print table information